# Lab 2: Analyzing Water Networks and Energy Use in CA

In this lab, we will be visualizing and analyzing water networks for specific utilities in CA. At this time you should have been assigned a utility to visualize and analyze. Here is a mapping of utilities to their respective codes in the graphs and data.

Utility Code|Utility Name
-|-
1805003E|Alameda County Water District
1803033E|Fresno City of
1810019E|Hi-Desert Water District
1809027E|Palmdale Water District
1805085E|San Jose Water Company
1807341E|Santa Monica City of

To start off the lab, we have to import some tools for us to be able to select and visualize certain utilities. These tools will build graphs for us which show which water sources and which types of water are upstream from a certain utility. The tools will also enable us to perform energy analysis on the utilities.Therefore, just run the cell and this will set up our environment.

In [1]:
from sub_network import subWESTnet
import pandas as pd
import numpy as np
import json, urllib, ast
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout

Here, we create what is called an instance of a graph object. This is what will enable us to be able to visualize the water network associated with a certain utility, all the sources and the types of water that are leading into it.

In [2]:
new_upstream_example = subWESTnet('data/fixed_times_erl.csv', 2010)

This line of code specifies that we want to look at all the sources upstream from the utility. The point of this is that we want to analyze what are the types of water coming into these utilities, how much is coming in, and from where is it coming. In the spot that has "Your Utility Name Here" put your utility name in the quotes. To see what your specific utility name is, look at the table at the top of this page and put that name specifically in the quotes.

In [8]:
new_upstream_example.upstream('San Jose Water Company')

Unweighted upstream graph built.


This line of code is here to compute all the amounts of water going into the different nodes in the network, all you have to do is run the following cell

In [9]:
upstream_example.balance_graph()

Graph is weighted.


In [10]:
new_upstream_example.balance_graph()

Graph is weighted.


The following attributes of our upstream and downstream examples show the tables of the actually network (by using source and target labels) and the energies associated with each node in the graph.

These two cells, take the downstream and upstream tables, and turn them into a dot file. The dotfile is placed in 

ouput/sub_(your_utility_code)_2010__upstream/downstream.dot

In [11]:
upstream_example.to_dot()

dot saved


In [12]:
new_upstream_example.to_dot()

dot saved


go to the dot file (inside the ouput directory where the lab2 notebook is located) and paste the code in the dot file [here](https://dreampuf.github.io/GraphvizOnline/). Make sure to replace the entire code block that is already in the link with the entire code block in our output.dot files.

In [6]:
def get_sankey_file(data_path, year, utility, upstream=True):
    gi = subWESTnet(data_path, year)
    gi.upstream(utility)
    
    sank_dicts = []
    for v in gi.edges.values():
        sank_dict = {}
        sank_dict['source'] = v['source']
        sank_dict['target'] = v['target']
        sank_dict['value'] = v['used_vol_af']
        sank_dict['color'] = 'steelblue'
        if v['used_vol_af'] == 0:
            sank_dict['color'] = 'goldenrod'
            sank_dict['value'] = .00001
        sank_dicts.append(sank_dict)
    
    df = pd.DataFrame(sank_dicts)
    df = df[df['color'] != 'goldenrod']
    data = pd.read_csv(data_path)
    
    nodes = []
    for i in df['source'].unique():
        sdict = {}
    
        # check if resource/end or not
        check_resource = data[data['target'] == i].shape
        if check_resource[0] == 0:
            sdict['is resource'] = True
        else:
            sdict['is resource'] = False
        if i[-1:] == 'E':
            sdict['is end'] = True
        else:
            sdict['is end'] = False
    
        # compute in/out volumes
        outv = df[df['source'] == i]['value'].sum()
        inv = df[df['target'] == i]['value'].sum()
    
        # get values to compensate for missing volumes
        extra_case = {}
        if outv < inv:
            if not sdict['is end']:
                extra_case['value'] = inv - outv
                extra_case['source'] = i
                extra_case['target'] = i + " to Other"
                nodes.append(extra_case)
        elif outv > inv:
            if not sdict['is resource']:
                extra_case['value'] = outv - inv
                extra_case['source'] = "Other to " + i
                extra_case['target'] = i
                nodes.append(extra_case)
    
    extra_ = df.drop('color', axis=1).append(pd.DataFrame(nodes)).reset_index(drop=True)
    a = []
    count = 0
    break_count = 0
    break_break = []
    while 'break' != break_break:
        drop = []
        for i in extra_['target']:
            if extra_[extra_['source'] == i].shape[0] == 0:
                if i[-5:] != 'Other' and i[-1:] != 'E':
                    drop.append(i)
        extra_ = extra_.loc[[k for k, v in extra_.iterrows() if v['target'] not in drop]]
        count += len(drop)
        a.extend(drop)
        if count == len(a):
            break_count += 1
        if break_count == 50:
            break_break = 'break'
    
    return extra_

SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=get_sankey_file('data/links_erl.csv', 2015, '1805085E', upstream=True).to_dict('records'))

Unweighted upstream graph built.


SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': '1805089PD', 'target': '1805085PD',…

In [7]:
SankeyWidget(layout=Layout(width="1500", height="1000"), 
             margins=dict(top=10, bottom=0, left=100, right=100),
             links=get_sankey_file('data/links_erl.csv', 2010, '1805085E', upstream=True).to_dict('records'))

Unweighted upstream graph built.


SankeyWidget(layout=Layout(height='1000', width='1500'), links=[{'source': '1805089PD', 'target': '1805085PD',…